In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
cd drive/My\ Drive/ML

/content/drive/My Drive/ML


In [0]:
import sklearn # general ml package
import math
import numpy as np
import pandas as pd # Pandas module to enable data manipulation
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

# Common imports
import numpy as np # fundamental package for scientific computing
import os # to run file I/O operation 

# to make this notebook's output stable across runs
# any number will do, as long as it is used consistently
np.random.seed(42)

In [0]:
ls

drive/  sample_data/


In [0]:
#crashes=pd.read_csv('new_Virginia_Crashes.csv')

In [0]:
crashes=pd.read_csv('Virginia_Crashes.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (45,67) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
junk = ['OBJECTID', 'Document_Nbr', 'Crash_Year', 'Passgen', 'Pedinjurytype', 'Pedage', 'Pedgen', 'DIAGRAM', 'VSP', 'MPO_NAME', 'A_Crash', 'B_Crash', 'C_Crash', 'Pdo_Crash',
        'Passage', 'Rte_Nm', 'Rns_Mp', 'Ownership', 'FUN', 'X', 'Y', 'Route_Or_Street_Nm', 'Local_Case_Cd', 'K_People', 'Pedestrians_Killed', 'Pedestrians_Injured', 'A_People', 'B_People',
        'C_People', 'Motor_Nonmotor', 'Crash_Military_Tm', 'Driverinjurytype', 'Node_Info', 'Pdo_Crash', 'Passage', 'Rte_Nm', 'Rns_Mp', 'Ownership', 'FUN', 'Rd_Type', 'Passinjurytype']
crashes = crashes.drop(junk, axis=1)

In [0]:
binaries = ['Fatal_Crashes', 'Crash_Severity', 'School_Zone', 'Ped_Nonped', 'Bike_Nonbike', 'Speed_Notspeed', 'Belted_Unbelted', 'Alcohol_Notalcohol', 'Distracted_Notdistracted', 'Senior_Notsenior', 'Young_Notyoung', 'Deer_Nodeer']
b_crashes = crashes[binaries]
crashes = crashes.drop(binaries, axis=1)

In [0]:
def nanp(x): #Returns 1 if the driver was distracted, 0 otherwise - Arthur
  try:
    if(math.isnan(x)):
      return 0
    else:
      return 1
  except:
    return 1
def Schoolp(x):
  if(x == '3. No'):
    return 0
  else:
    return 1
def pedp(x):
  if(x == 'NOTPED'):
    return 0
  else:
    return 1


In [0]:
b_crashes['Crash_Severity'].value_counts()

PDO.Property Damage Only    586308
B.Visible Injury            174736
C.Nonvisible Injury          82540
A.Severe Injury              44907
K.Fatal Injury                5172
Name: Crash_Severity, dtype: int64

In [0]:
def severity_categories(x):
    if x == 'PDO.Property Damage Only':
      return 0
    if x == 'B.Visible Injury':
      return 1
    if x == 'C.Nonvisible Injury':
      return 2
    if x == 'A.Severe Injury':
      return 3
    if x == 'K.Fatal Injury':
      return 4
    else:
      return 9

In [0]:
b_crashes.Crash_Severity = b_crashes.Crash_Severity.apply(lambda x : severity_categories(x))

In [0]:
a.value_counts()

0    586308
1    174736
2     82540
3     44907
4      5172
Name: Crash_Severity, dtype: int64

In [0]:
def severity_groupings(x):
  if x == 'PDO.Property Damage Only':
    return 0
  else:
    return 1

In [0]:
#b_crashes.Crash_Severity = b_crashes.Crash_Severity.apply(lambda x : severity_groupings(x))
b_crashes['Bike_Nonbike'] = b_crashes['Bike_Nonbike'].apply(lambda x : nanp(x))
b_crashes.Speed_Notspeed = b_crashes.Speed_Notspeed.apply(lambda x  : nanp(x))
b_crashes.Belted_Unbelted = b_crashes.Belted_Unbelted.apply(lambda x  : nanp(x))
b_crashes.Alcohol_Notalcohol = b_crashes.Alcohol_Notalcohol.apply(lambda x  : nanp(x))
b_crashes.Distracted_Notdistracted = b_crashes.Distracted_Notdistracted.apply(lambda x : nanp(x))
b_crashes.Senior_Notsenior = b_crashes.Senior_Notsenior.apply(lambda x : nanp(x))
b_crashes.Young_Notyoung = b_crashes.Young_Notyoung.apply(lambda x : nanp(x))
b_crashes.Deer_Nodeer = b_crashes.Deer_Nodeer.apply(lambda x : nanp(x))
b_crashes.School_Zone = b_crashes.School_Zone.apply(lambda x : Schoolp(x))
b_crashes.Ped_Nonped = b_crashes.Ped_Nonped.apply(lambda x : pedp(x))

In [0]:
crashes.FAC = crashes.FAC.fillna('0.Two-way, non-divided')
crashes['Driverage'] = crashes['Driverage'].apply(lambda x : str(x).split(',')[0]) #Defines Driver age to be the first one listed, arbitrary but better solution is unclear
crashes['Driverage'].replace('nan', np.nan, inplace = True)
crashes['Driverage'] = crashes['Driverage'].apply(lambda x : float(x))
crashes['Driverage'].replace(np.nan, np.mean(crashes['Driverage']), inplace= True) #replace nans with mean
def month(x):
  try:
    return x.split('-')[1]
  except:
    return 'None'
crashes.Crash_Dt = crashes.Crash_Dt.apply(lambda x: month(x))
crashes['Vehiclenumber'] = crashes['Vehiclenumber'].apply(str)
crashes['Vehiclenumber'] = crashes['Vehiclenumber'].apply(lambda x : int(x.split(',')[-1]))
crashes['Vehicle_Body_Type_Cd'] = crashes['Vehicle_Body_Type_Cd'].apply(lambda x : (x.split(',')[-1]))

In [0]:
crashes.Driver_Action_Type_Cd = crashes.Driver_Action_Type_Cd.apply(lambda x : (x.split(',')[-1]))

In [0]:
crashes.Drivergen = crashes.Drivergen.apply(lambda x : (x.split(',')[-1]))

In [0]:
cols = crashes.columns

In [0]:
cat_cols =['Collision_Type', 'Roadway_Surface_Cond',
       'First_Harmful_Event_of_Entire', 'Weather_Condition', 'Light_Condition',
       'Crash_Event_Type_Dsc', 'Time_Slicing',
       'Vehicle_Body_Type_Cd', 'Drivergen',
       'Driver_Action_Type_Cd', 'SYSTEM', 'FAC',
       'VDOT_District',
       'Crash_Dt']
other = [
       'Vehiclenumber',
       'Driverage',
       'Carspeedlimit', 'LATITUDE',
       'LONGITUDE', 'Physical_Juris', 'PLAN_DISTRICT']

In [0]:
crashes[cat_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 893663 entries, 0 to 893662
Data columns (total 14 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   Collision_Type                 893663 non-null  object
 1   Roadway_Surface_Cond           893663 non-null  object
 2   First_Harmful_Event_of_Entire  893663 non-null  object
 3   Weather_Condition              893663 non-null  object
 4   Light_Condition                893663 non-null  object
 5   Crash_Event_Type_Dsc           893663 non-null  object
 6   Time_Slicing                   893663 non-null  object
 7   Vehicle_Body_Type_Cd           893663 non-null  object
 8   Drivergen                      893663 non-null  object
 9   Driver_Action_Type_Cd          893663 non-null  object
 10  SYSTEM                         893663 non-null  object
 11  FAC                            893663 non-null  object
 12  VDOT_District                  893663 non-nu

In [0]:
final = pd.concat([b_crashes, pd.get_dummies(crashes[cat_cols]), crashes[other]], axis=1)

In [0]:
final.Crash_Severity.value_counts()

0    586308
1    174736
2     82540
3     44907
4      5172
Name: Crash_Severity, dtype: int64

In [0]:
final.to_csv('clean_data_5.csv')

In [0]:
final.columns = ['Injury' if x=='Crash_Severity' else x for x in final.columns]

In [0]:
final['Injury'].value_counts()

0    586308
1    307355
Name: Injury, dtype: int64

In [0]:
final.to_csv('clean_data.csv')

In [0]:
final

Fatal_Crashes  ...                  PLAN_DISTRICT
0                   0  ...  16.George Washington Regional
1                   0  ...               23.Hampton Roads
2                   0  ...            8.Northern Virginia
3                   0  ...               23.Hampton Roads
4                   0  ...     5.Roanoke Valley-Alleghany
...               ...  ...                            ...
893658              0  ...               23.Hampton Roads
893659              0  ...               23.Hampton Roads
893660              0  ...            8.Northern Virginia
893661              0  ...            8.Northern Virginia
893662              0  ...  16.George Washington Regional

[893663 rows x 230 columns]

In [0]:
final.shape

In [0]:
final.head()

In [0]:
ls

In [0]:
crash2 = pd.read_csv("clean_data.csv")

In [0]:
crash2.head()

Unnamed: 0  Fatal_Crashes  ...      Physical_Juris                  PLAN_DISTRICT
0           0              0  ...     88.Spotsylvania  16.George Washington Regional
1           1              0  ...  133.Elizabeth City               23.Hampton Roads
2           2              0  ...          53.Loudoun            8.Northern Virginia
3           3              0  ...         114.Hampton               23.Hampton Roads
4           4              0  ...           129.Salem     5.Roanoke Valley-Alleghany

[5 rows x 231 columns]

In [0]:
arr = final.drop(['Physical_Juris', 'PLAN_DISTRICT', 'Fatal_Crashes'], axis=1).to_numpy()

In [0]:
y = final['Fatal_Crashes'].to_numpy()

In [0]:
arr.shape

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(arr, y, random_state=42)
#rnd_clf = RandomForestClassifier(n_estimators=50, random_state=42)
X_train_network, X_val_network, y_train_network, y_val_network = train_test_split(X_train, y_train, random_state=42, train_size = 0.75)

In [0]:
model2 = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(50, activation='relu'),
    #keras.layers.Dropout(0.5),
    #keras.layers.Dense(40, activation='relu'),
    #keras.layers.Dropout(0.5),
    #keras.layers.Dense(60, activation='relu'),
    #keras.layers.Dropout(0.5),
    #keras.layers.Dense(400, activation='relu'),
    #keras.layers.Dropout(0.5),
    #keras.layers.Dense(200, activation='relu'),
    #keras.layers.Dropout(0.5),
    keras.layers.Dense(units=1, activation='softmax'),
])
model2.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=['accuracy', keras.metrics.Recall(), keras.metrics.Precision()])

In [0]:
model2.fit(X_train_network, y_train_network, validation_data=(X_val_network, y_val_network), epochs=20)

Epoch 1/20


ValueError: ignored

In [0]:
crash3 = pd.read_csv("clean_data.csv")

In [0]:
crash3

Unnamed: 0  ...                  PLAN_DISTRICT
0                0  ...  16.George Washington Regional
1                1  ...               23.Hampton Roads
2                2  ...            8.Northern Virginia
3                3  ...               23.Hampton Roads
4                4  ...     5.Roanoke Valley-Alleghany
...            ...  ...                            ...
893658      893658  ...               23.Hampton Roads
893659      893659  ...               23.Hampton Roads
893660      893660  ...            8.Northern Virginia
893661      893661  ...            8.Northern Virginia
893662      893662  ...  16.George Washington Regional

[893663 rows x 230 columns]

In [0]:
orig = pd.read_csv("Virginia_Crashes.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (41,42,43,45,48,49) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
orig

X          Y  ...  Young_Notyoung   VDOT_District_1
0      -77.200260  36.715610  ...       Not_YOUNG   5.Hampton Roads
1      -77.568560  38.527250  ...       Not_YOUNG        7.Culpeper
2      -78.516050  38.273410  ...       Not_YOUNG        7.Culpeper
3      -77.956010  38.317450  ...       Not_YOUNG        7.Culpeper
4      -77.523967  38.220247  ...       Not_YOUNG  6.Fredericksburg
...           ...        ...  ...             ...               ...
859967 -76.425420  37.201967  ...       Not_YOUNG   5.Hampton Roads
859968 -76.762120  37.314390  ...       Not_YOUNG   5.Hampton Roads
859969 -76.450860  37.151100  ...       Not_YOUNG   5.Hampton Roads
859970 -76.429450  37.105260  ...       Not_YOUNG   5.Hampton Roads
859971 -76.650380  36.945970  ...       Not_YOUNG   5.Hampton Roads

[859972 rows x 66 columns]